In [28]:
import numpy as np
from scipy.sparse import csc_matrix, lil_matrix
from scipy.stats import pearsonr
import random

from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd

In [29]:
# 4
# Loading comments.csv

comments_csv = pd.read_csv("comments.csv", delimiter= ',', usecols=[0, 1, 4])

In [30]:
# 4
# Generate item-user matrix from comments

unique_users = comments_csv["personId"].unique()
unique_items = comments_csv["contentId"].unique()

item_user_matrix = lil_matrix((len(unique_items), len(unique_users)), dtype=float)

vader = SentimentIntensityAnalyzer()

for index, row in comments_csv.iterrows():
	item_index = np.where(unique_items == row["contentId"])[0][0]
	user_index = np.where(unique_users == row["personId"])[0][0]
	item_user_matrix[item_index, user_index] = vader.polarity_scores(row["comment"])["compound"]

item_user_matrix = item_user_matrix.tocsc()

test_vec = [random.randint(0, 5) for _ in range(len(unique_items))]
test_vec2 = item_user_matrix.getcol(3).toarray().flatten()

corr, _ = pearsonr(test_vec, test_vec2)
print(corr) # Correlation with a random vector should be pretty close to 0

262
568
-0.0487347170495082
